In [1]:
import pandas as pd 
import numpy as np
from collections import defaultdict

In [6]:
df = pd.read_csv(r"C:\Users\araca\Documents\UNIVERSIDAD\PROYECTO DE DATOS I\df_limpio.csv")
df


,Unnamed: 0,Peleador_A,Peleador_B,DATE,CATEGORY,WINNER,METHOD,TIME,ROUND,KD_A,...,STR_CLINCH_A_x,STR_CLINCH_A_y,STR_CLINCH_B_x,STR_CLINCH_B_y,STR_GROUND_A_x,STR_GROUND_A_y,STR_GROUND_B_x,STR_GROUND_B_y,TITLE_FIGHT,GENDER
0,0,Jon Jones,Glover Teixeira,"April 26, 2014",LIGHT HEAVYWEIGHT,0,Decision - Unanimous,300,5,0,...,37,49,23,57,2,5,0,0,True,0
1,1,Phil Davis,Anthony Johnson,"April 26, 2014",LIGHT HEAVYWEIGHT,1,Decision - Unanimous,300,3,0,...,3,4,3,6,2,2,3,5,False,0
2,2,Luke Rockhold,Tim Boetsch,"April 26, 2014",MIDDLEWEIGHT,0,Submission,128,1,0,...,0,0,0,0,3,3,0,0,False,0
3,3,Jim Miller,Yancy Medeiros,"April 26, 2014",LIGHTWEIGHT,0,Submission,198,1,0,...,1,2,1,1,0,0,0,0,False,0
4,4,Max Holloway,Andre Fili,"April 26, 2014",FEATHERWEIGHT,0,Submission,219,3,0,...,17,28,4,10,0,0,0,0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7839,7839,Yan Cabral,Zak Cummings,"May 10, 2014",WELTERWEIGHT,1,Decision - Unanimous,300,3,0,...,0,0,3,5,15,15,13,25,False,0
7840,7840,Eddie Wineland,Johnny Eduardo,"May 10, 2014",BANTAMWEIGHT,1,KO/TKO,277,1,0,...,0,0,1,1,0,0,0,0,False,0
7841,7841,Manvel Gamburyan,Nik Lentz,"May 10, 2014",FEATHERWEIGHT,1,Decision - Unanimous,300,3,0,...,18,22,25,34,0,2,7,10,False,0
7842,7842,Justin Salas,Ben Wall,"May 10, 2014",LIGHTWEIGHT,0,KO/TKO,161,1,1,...,7,8,3,3,6,6,0,0,False,0


In [7]:
df["METHOD"].unique()

array(['Decision - Unanimous', 'Submission', 'KO/TKO', 'Decision - Split',
       'Decision - Majority', 'Could Not Continue', 'Overturned',
       "TKO - Doctor's Stoppage", 'DQ'], dtype=object)

In [31]:
# Inicializar diccionario de puntuaciones
ranking = defaultdict(lambda: 100)  # Empezamos con 100 puntos por peleador
peleas = defaultdict(int)
victorias = defaultdict(int)
derrotas = defaultdict(int)
racha_win = defaultdict(int)
racha_loss = defaultdict(int)


# Definir pesos para los métodos de victoria
pesos = {"KO/TKO" or "TKO": 1, "Submission": 0.75, "Decision - Unanimous" or "Decision - Majority": 0.5}

def actualizar_puntuacion(ganador, perdedor, metodo):
    peso = pesos.get(metodo, 0.25)  # Por defecto, 0.5 si el método no está especificado
    factor = ranking[perdedor] / 100  # Se pondera según el nivel del rival 
    peleas[ganador] += 1
    victorias[ganador]+=1
    derrotas[perdedor]+=1
    peleas[perdedor] += 1
    racha_win[ganador]+=1
    racha_win[perdedor]=0
    racha_loss[ganador]=0
    racha_loss[perdedor]+=1

    ranking[ganador] += peso * factor * 10  * (1+victorias[ganador]/peleas[ganador]) * racha_win[ganador] # Se ajusta la puntuación
    ranking[perdedor] -= (1 - peso) * factor * 5 * (1+ derrotas[perdedor]/peleas[perdedor]) * racha_loss[perdedor]# Penalización por perder

# Iterar sobre cada pelea y actualizar el ranking
for _, row in reversed(list(df.iterrows())):
    if row['WINNER'] == 0:
        actualizar_puntuacion(row['Peleador_A'], row['Peleador_B'], row['METHOD'])
    else:
        actualizar_puntuacion(row['Peleador_B'], row['Peleador_A'], row['METHOD'])

df_ranking = pd.DataFrame(
    sorted(ranking.items(), key=lambda x: x[1], reverse=True), 
    columns=['Peleador', 'Puntuación']
)

# Agregar la columna de peleas
df_ranking['Peleas'] = df_ranking['Peleador'].map(peleas)
df_ranking['Racha'] = df_ranking['Peleador'].map(racha_win)


# Mostrar el top 10
df_ranking.head(20)


,Peleador,Puntuación,Peleas,Racha
0,Islam Makhachev,52455.795394,17,15
1,Jon Jones,29503.862383,24,11
2,Chris Weidman,22442.605126,20,7
3,Ilia Topuria,17397.187663,8,8
4,Dricus Du Plessis,15258.866739,9,9
5,Belal Muhammad,14939.528772,19,11
6,Kamaru Usman,11648.426972,18,0
7,Merab Dvalishvili,11485.952576,14,12
8,Khabib Nurmagomedov,11400.692916,13,13
9,Charles Oliveira,10271.512713,34,1
